In [2]:
#Import things
import astropy
import astroquery
from astroquery.gaia import Gaia
from astropy.coordinates import SkyCoord
from astropy.coordinates import Angle
import astropy.units as u
import numpy as np


In [3]:

# Gaia Query

coord = SkyCoord(ra=280, dec=-60, unit=(u.degree, u.degree), frame='icrs')
width = u.Quantity(0.1, u.deg)
height = u.Quantity(0.1, u.deg)
r = Gaia.query_object_async(coordinate=coord, width=width, height=height)

r.pprint(max_lines=12, max_width=130)
print(type(r))

INFO: Query finished. [astroquery.utils.tap.core]
         dist             solution_id     ...                                     datalink_url                                   
                                          ...                                                                                    
--------------------- ------------------- ... -----------------------------------------------------------------------------------
0.0026034636994048854 1635721458409799680 ... https://gea.esac.esa.int/data-server/datalink/links?ID=Gaia+DR2+6636090334814214528
0.0038518741347606357 1635721458409799680 ... https://gea.esac.esa.int/data-server/datalink/links?ID=Gaia+DR2+6636090339113063296
  0.00454542650096783 1635721458409799680 ... https://gea.esac.esa.int/data-server/datalink/links?ID=Gaia+DR2+6636090334814217600
                  ...                 ... ...                                                                                 ...
 0.020307185970548904 163572145840979968

In [4]:
# ATNF Query
from psrqpy import QueryATNF
query = QueryATNF(condition= 'PMRA > 0')
table = query.table
print(table)

    PSRJ    PSRJ_REF        RAJ        ... TYPE_ORIG  TYPE_REF  BINCOMP_ORIG
                                       ...                                  
----------- -------- ----------------- ... ---------- -------- -------------
 J0014+4746       --       00:14:17.75 ...         --                     --
J0024-7204C       --     00:23:50.3546 ...         --                     --
J0024-7204D       --    00:24:13.88092 ...         --                     --
J0024-7204E       --    00:24:11.10528 ...         --                     --
J0024-7204F       --    00:24:03.85547 ...         --                     --
J0024-7204G       --     00:24:07.9603 ...         --                     --
J0024-7204H       --     00:24:06.7032 ...         --             He[fck+03]
J0024-7204I       --     00:24:07.9347 ...         --             UL[fck+03]
J0024-7204J       --     00:23:59.4077 ...         --             UL[fck+03]
J0024-7204L       --     00:24:03.7721 ...         --                     --

In [5]:
print(table.info)

<Table length=201>
    name      dtype       unit         class     n_bad
------------ ------- -------------- ------------ -----
        PSRJ   str12                      Column     0
    PSRJ_REF  object                MaskedColumn   106
         RAJ   str17                      Column     0
     RAJ_ERR float64                MaskedColumn     1
     RAJ_REF    str7                      Column     0
        DECJ   str18                      Column     0
    DECJ_ERR float64                      Column     0
    DECJ_REF    str7                      Column     0
    POSEPOCH float64              d       Column     0
POSEPOCH_REF  object                MaskedColumn     3
          F0 float64             Hz       Column     0
      F0_ERR float64             Hz MaskedColumn     1
      F0_REF    str7                      Column     0
          F1 float64         1 / s2 MaskedColumn     1
      F1_ERR float64         1 / s2 MaskedColumn     1
      F1_REF    str7                      Colu

Given an item from ATNF catalog, find nearby Gaia objects

In [6]:
from astropy.time import Time

# Gather information on a given pulsar

pulsar = 1 # check first pulsar listed in ATNF

p_ascension = table["RAJ"][pulsar]
p_declination = table["DECJ"][pulsar]
p_pmra = table["PMRA"][pulsar]
p_pmdec = table["PMDEC"][pulsar]

print (p_ascension + " " + p_declination)

raw_epoch = table["POSEPOCH"][pulsar]
p_epoch = Time(raw_epoch, format='mjd').jyear

print(p_epoch) # testing that p_epoch is now in the correct time units, jyear

p_ascension_angle = Angle(p_ascension.tolist(), u.degree) # convert p_ascension to Angle object 
p_ascension_degree = p_ascension_angle * u.deg # convert to degree units (float64)


p_declination_angle = Angle(p_declination.tolist(), u.degree) # convert p_declination to Angle object
p_declination_degree = p_declination_angle * u.deg # convert to degree units (float64)


print(type(p_declination_angle))
print(type(p_epoch.tolist()))
print(p_declination_angle.unit)
print(table["JNAME"][pulsar])


00:23:50.3546 -72:04:31.5048
2000.1519507186858
<class 'astropy.coordinates.angles.Angle'>
<class 'float'>
deg
J0024-7204C


In [7]:
gaia_epoch = 2015.5 * u.yr # define gaia epoch in units years
year_diff = gaia_epoch - p_epoch.tolist() * u.yr # define the time difference in years as a float

p_pmra_deg = p_pmra.tolist() * (u.degree / u.yr) # convert p_pmra from mas/yr to deg/yr
p_pmdec_deg = p_pmdec.tolist() * (u.degree /u.yr) # convert p_pmdec_deg from mas/yr to deg/yr
assert p_pmra_deg.unit.is_equivalent(u.degree / u.yr)
assert p_pmdec_deg.unit.is_equivalent(u.degree / u.yr)
assert year_diff.unit.is_equivalent(u.yr)
print((p_pmra_deg*year_diff).unit)
print(p_ascension_degree.unit)

deg
deg2


In [8]:
# Update location of pulsar based on epoch difference with Gaia

gaia_epoch = 2015.5 * u.yr # define gaia epoch in units years
year_diff = gaia_epoch - p_epoch.tolist() * u.yr # define the time difference in years as a float
p_new_ascension = p_ascension_angle + (p_pmra_deg * year_diff)
p_new_declination = p_declination_angle + (p_pmdec_deg * year_diff)

print(p_new_ascension + " " + p_new_declination)

TypeError: Unsupported operand type(s) for ufunc add: 'Angle,str'

In [ ]:
# Querying Gaia

coord = SkyCoord(ra=280, dec=-60, unit=(u.degree, u.degree), frame='icrs')
width = u.Quantity(0.1, u.deg)
height = u.Quantity(0.1, u.deg)
results = Gaia.query_object_async(coordinate=coord, width=width, height=height)
results.show_in_notebook()

INFO: Query finished. [astroquery.utils.tap.core]


idx,dist,solution_id,DESIGNATION,source_id,random_index,ref_epoch,ra,ra_error,dec,dec_error,parallax,parallax_error,parallax_over_error,pmra,pmra_error,pmdec,pmdec_error,ra_dec_corr,ra_parallax_corr,ra_pmra_corr,ra_pmdec_corr,dec_parallax_corr,dec_pmra_corr,dec_pmdec_corr,parallax_pmra_corr,parallax_pmdec_corr,pmra_pmdec_corr,astrometric_n_obs_al,astrometric_n_obs_ac,astrometric_n_good_obs_al,astrometric_n_bad_obs_al,astrometric_gof_al,astrometric_chi2_al,astrometric_excess_noise,astrometric_excess_noise_sig,astrometric_params_solved,astrometric_primary_flag,astrometric_weight_al,astrometric_pseudo_colour,astrometric_pseudo_colour_error,mean_varpi_factor_al,astrometric_matched_observations,visibility_periods_used,astrometric_sigma5d_max,frame_rotator_object_type,matched_observations,duplicated_source,phot_g_n_obs,phot_g_mean_flux,phot_g_mean_flux_error,phot_g_mean_flux_over_error,phot_g_mean_mag,phot_bp_n_obs,phot_bp_mean_flux,phot_bp_mean_flux_error,phot_bp_mean_flux_over_error,phot_bp_mean_mag,phot_rp_n_obs,phot_rp_mean_flux,phot_rp_mean_flux_error,phot_rp_mean_flux_over_error,phot_rp_mean_mag,phot_bp_rp_excess_factor,phot_proc_mode,bp_rp,bp_g,g_rp,radial_velocity,radial_velocity_error,rv_nb_transits,rv_template_teff,rv_template_logg,rv_template_fe_h,phot_variable_flag,l,b,ecl_lon,ecl_lat,priam_flags,teff_val,teff_percentile_lower,teff_percentile_upper,a_g_val,a_g_percentile_lower,a_g_percentile_upper,e_bp_min_rp_val,e_bp_min_rp_percentile_lower,e_bp_min_rp_percentile_upper,flame_flags,radius_val,radius_percentile_lower,radius_percentile_upper,lum_val,lum_percentile_lower,lum_percentile_upper,datalink_url
,,,,,,yr,deg,mas,deg,mas,mas,mas,,mas / yr,mas / yr,mas / yr,mas / yr,,,,,,,,,,,,,,,,,mas,,,,1 / mas2,1 / um,1 / um,,,,mas,,,,,electron / s,electron / s,,mag,,electron / s,electron / s,,mag,,electron / s,electron / s,,mag,,,mag,mag,mag,km / s,km / s,,K,log(cm.s**-2),dex,,deg,deg,deg,deg,,K,K,K,mag,mag,mag,mag,mag,mag,,solRad,solRad,solRad,solLum,solLum,solLum,
0,0.0026034636994048854,1635721458409799680,Gaia DR2 6636090334814214528,6636090334814214528,1176213019,2015.5,280.0002535109784,0.29861262266219235,-60.00259470935402,0.3286090424553563,-0.3110984894867886,0.4659006859423401,-0.66773564,-0.3829646819799206,0.5485869545050303,-6.494624923325772,0.5631744832783001,0.14009339,-0.032677036,-0.040756915,0.1848471,-0.24814793,0.25431943,-0.020390274,-0.41731763,-0.023678254,0.053505838,273,0,272,1,2.0634625,316.8305,0.8687575596881527,0.8324284930248446,31,False,0.08247371,1.5682945669210009,0.10173123960452841,0.09272486,31,14,0.5277922,0,32,False,277,229.27686620684315,1.1770479415122068,194.78975,19.787466,29,132.09798809841195,9.236134667236668,14.302302,20.049149,25,169.83721262885817,6.933962725221738,24.493528,19.186838,1.3169022,0,0.8623104,0.2616825,0.6006279,--,--,0,--,--,--,NOT_AVAILABLE,335.4869899853897,-21.882743330467846,276.222852405146,-36.779152165694725,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,https://gea.esac.esa.int/data-server/datalink/links?ID=Gaia+DR2+6636090334814214528
1,0.0038518741347606357,1635721458409799680,Gaia DR2 6636090339113063296,6636090339113063296,570813457,2015.5,280.0051165709225,0.050240754418698456,-59.9971106177236,0.05569779779421987,2.1445897979735338,0.08686626499655296,24.688408,-30.239136112439823,0.09715628380624572,7.29370185769282,0.0908715196903835,-0.0498363,0.11446786,-0.18721049,0.35760036,-0.4172074,0.47041324,-0.055835903,-0.5549303,0.14921485,-0.123301156,264,0,262,2,2.5903792,319.54343,0.2221438529687765,2.164099682113238,31,False,3.5670664,1.4180211041511326,0.015143605381231719,0.07440901,30,14,0.10592062,0,31,False,256,4190.134904536919,3.2053909995733343,1307.215,16.632795,27,1280.5501396270038,8.934403906995241,143.32799,17.582897,25,4354.193392244988,10.443625853844335,416.92352,15.664651,1.3447642,1,1.9182463,0.95010185,0.9681444,--,--,0,--,--,--,NOT_AVAILABLE,335.49340340182914,-21.883498064512462,276.2264674062974,-36.773898257890345,100002,3817